In [1]:
import OffsetArrays
using Plots
using Revise
using StatsBase
using ProgressMeter
using SparseArrays, LinearAlgebra, IndexedGraphs, Graphs
using Distributions
srcpath = "./src"
include("$srcpath/bp.jl") #julia code which contains the bp update
include("$srcpath/post_infer.jl") #definition of the main functions and the types
include("$srcpath/observables.jl") # julia code for estimating observables

avg_ninf (generic function with 1 method)

In [3]:
λp = 1.0# planted infection rate
λi = λp #inferred infection rate

T = 8 # discrete time: number of time-steps

γp = 0.1 #0.019 # planted autoinfection probability
γi = γp # inferred autoinfection probability

N = 5000; #population size
@show 2/sqrt(N)

dilution = 0.0 #dilution of observations. dil=0 means everybody observed once
fr = 0.0; #noise in the observation. 

degree_dist = Dirac(3) #the distribution of the degree in the graph. Dirac means random regular.

2 / sqrt(N) = 0.0282842712474619


Dirac{Int64}(value=3)

In [4]:
M_zero = ParametricModel(N=N,T=T,γp=γp,λp=λp,γi=γi,λi=λi,fr=fr,dilution=dilution,distribution=degree_dist);

In [5]:
F_zero,it_zero = pop_dynamics(M_zero)#, tot_iterations = 100);

#1.iter 2.err 3.F
0	-1	0.05500729680944642
1	-1	-0.4664896742362471
2	-1	-0.39488339016772805
3	-1	-0.3910153514253878
4	-1	-0.3910143954716725


([0.05500729680944642, -0.4664896742362471, -0.39488339016772805, -0.3910153514253878, -0.3910143954716725, 0.0, 0.0, 0.0, 0.0, 0.0], 5)

In [26]:
#M_zero.mom1μ
#sum(M_zero.mom1μ)./length(M_zero.mom1μ)

In [27]:
#marg_zero = M_zero.belief |> real;
#marg2D_zero = reshape(sum(marg_zero,dims=3) ./ N, T+2,T+2)
#ensAUC_zero = avgAUC(marg_zero,M_zero.obs_list,count_obs=true);

In [28]:
M = ParametricModel(N=N,T=T,γp=γp,λp=λp,γi=γi,λi=λi,fr=fr,dilution=dilution,distribution=degree_dist);

In [29]:
F,it = pop_dynamics_stab(M, tot_iterations = 100, nbstab=40);

#1.iter 2.err 3.F 4. Δ
0	-1	-0.1494314783501977	-1.0
1	-1	0.35732613110890377	-1.0
2	-1	0.38864305083316286	-1.0
3	-1	-0.17889255205963492	-1.0
4	-1	-0.3001647265281499	-1.0
5	-1	-0.04136481428438128	-1.0
6	-1	-0.3474460803118272	-1.0
7	-1	-2.5950466438387476	-1.0
8	-1	-8.495223530776036	-1.0
9	-1	-8.81425495837531	-1.0
10	-1	3.9170513067077555	-1.0
11	5.3887795172730595	11.001943296682422	-1.0
12	5.699708198558274	5.625581453204289	-1.0
13	5.709761052575355	1.1367138064396247	-1.0
14	5.716467302672482	-0.8743136846284338	-1.0
15	5.727755697153443	-1.1408408445867912	-1.0
16	5.739042869184094	-1.2678982893371604	-1.0
17	5.731510510267002	-2.41215928187276	-1.0
18	5.06159846278217	-1.6536072350307456	-1.0
19	4.022400167497388	-0.2543244581706589	-1.0
20	3.971445988440344	2.931063542207383	-1.0
21	2.408033354922075	2.5053139600323395	-1.0
22	1.699543559630942	0.6682832210341614	-1.0
23	1.660174835432859	0.3117474263187556	-1.0
24	1.8272497467061481	-2.7140681732159164	-1.0
25	2.260761936

LoadError: MethodError: no method matching log(::Nothing)
[0mClosest candidates are:
[0m  log([91m::StridedMatrix{T} where T[39m) at ~/dev/julia-1.8.5/share/julia/stdlib/v1.8/LinearAlgebra/src/dense.jl:774
[0m  log([91m::AbstractGray[39m) at ~/.julia/packages/ColorVectorSpace/QI5vM/src/ColorVectorSpace.jl:315
[0m  log([91m::UniformScaling[39m) at ~/dev/julia-1.8.5/share/julia/stdlib/v1.8/LinearAlgebra/src/uniformscaling.jl:173
[0m  ...

In [16]:
test = rand(-1:2:1, T+2, 2, T+2, 3, N).*0.00001;
mean(test), mean(M.μ), mean(M.mom1μ)

(6.0866666666666634e-9, 0.0016666666666666668, 4.324331667368392e-9)

In [17]:
extrema(test), extrema(M.μ), extrema(M.mom1μ)

((-1.0e-5, 1.0e-5), (0.0, 0.08090472429788192), (-1.334582163922407e-7, 4.109041142782289e-7))

In [18]:
std(test), std(M.μ), std(M.mom1μ)

(9.999999814291046e-6, 0.0029692850196844913, 1.1095832860748813e-8)

(-7.746640579475731e-16, 2.0479244851529683e-15)

In [ ]:
marg = M.belief |> real;
marg2D = reshape(sum(marg,dims=3) ./ N, T+2,T+2)
ensAUC = avgAUC(marg,M.obs_list,count_obs=true); #set false to see the AUC only on NON observed indiv.

In [ ]:
sum(abs.(marg2D.-marg2D_zero))/length(marg2D)

In [ ]:
scatter(ensAUC)
scatter!(ensAUC_zero)

In [ ]:
pheat = heatmap((marg2D),c=[:white,:blue,:black],xlabel="planted",ylabel="inferred",title="Average 2D marginal")
pheat_stab = heatmap((marg2D_zero.-marg2D),c=[:white,:blue,:black],xlabel="planted",ylabel="inferred")
plot(pheat,pheat_stab)

In [ ]:
scatter(F)
scatter!(F_zero)
hline!([0])